In [1]:
# import dependencies
import pandas as pd
import numerapi, os, datetime,time,gc
import sklearn.linear_model
from sklearn.linear_model import Lasso
from pycaret.regression import *

today = datetime.date.today()
dates = '2021-01-17'

In [2]:
#https://forum.numer.ai/t/reducing-memory/313
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

training_data = reduce_mem_usage(
    pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz"))
tournament_data = reduce_mem_usage(
    pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz"))

Memory usage after optimization is: 309.15 MB
Decreased by 74.3%
Memory usage after optimization is: 1069.75 MB
Decreased by 74.3%


In [3]:
#training_data = pd.read_csv('./csv/numerai_train_data.csv')#.set_index("id")
#tournament_data = pd.read_csv('./csv/numerai_tournament_data.csv')#.set_index('id')
TARGET_NAME = [x for x in training_data.columns if 'target' in x][0]
PREDICTION_NAME = "prediction"
feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]
#training_features = training_data[feature_cols]

In [4]:
'''
def dataframe_cast_to_float32(df, features):
    typemap = {}
    for f in features:
        typemap[f] = 'float32'
    return df.astype(typemap)
'''

"\ndef dataframe_cast_to_float32(df, features):\n    typemap = {}\n    for f in features:\n        typemap[f] = 'float32'\n    return df.astype(typemap)\n"

In [5]:
# Submissions are scored by spearman correlation
def correlation(predictions, targets):
    ranked_preds = predictions.rank(pct=True, method="first")
    return np.corrcoef(ranked_preds, targets)[0, 1]

# convenience method for scoring
def score(df):
    return correlation(df[PREDICTION_NAME], df[TARGET_NAME])

#training_data = dataframe_cast_to_float32(training_data, feature_cols)
#tournament_data = dataframe_cast_to_float32(tournament_data, feature_cols)

In [6]:
# Get your API keys and model_id from https://numer.ai/submit
public_id = "TIFYUYGPJCFQZT4SBLNYIOZWQWIMC2EQ"
secret_key = "QGLW6MHS7QIDPEELKVJ6DKUGBWHQMN7O3A2BCIV7CU4QKGTNVBZ7F5RRFF75M4DB"
model_id = "45b2b9e3-ed1b-4d82-bb2c-84f828b403fe"
model_2 = 'f624d92f-3965-4242-b069-dda40993fffa'
model_3 ='b58e9477-feb6-4cd3-ae53-290326a41ecd'
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

# n11

In [7]:
lists = [0.0007] 
for i in lists:
    print(i)

0.0007


In [8]:
n_of_features =[]
for i in lists:
    lasso = Lasso(alpha=i).fit(training_data[feature_cols],training_data['target'])
    sum_features = np.sum(lasso.coef_ !=0)
    n_of_features.append([i,sum_features])
    #n_of_features.append(sum_features)

n_of_features
print('n_of_features: ',n_of_features)
selected_features = []
for i,j in enumerate(lasso.coef_):
    if j !=0:
        selected_features.append(training_data.columns[i])

import pandas as pd
from xgboost import XGBRegressor

gc.collect()

predictions = []
for itr in range(30):
    model = XGBRegressor(max_depth=5, learning_rate=0.01,\
                         n_estimators=2000, n_jobs=-1, colsample_bytree=0.1, \
                         seed=itr, tree_method='gpu_hist')
    model.fit(training_data[selected_features], training_data[TARGET_NAME])
    predictions.append(model.predict(tournament_data[selected_features]))

prediction_df = tournament_data["id"].to_frame()
prediction_df.fillna(0,inplace=True)
tournament_data[TARGET_NAME].fillna(0,inplace=True)
scores = []
mmc =[]
for prediction in predictions:
    scores.append(correlation(pd.Series(prediction), tournament_data[TARGET_NAME]))
sum(scores)/len(scores)

prediction_df = tournament_data["id"].to_frame()
for itr in range(len(predictions)):
    prediction_df[f"prediction_{itr}"] = predictions[itr]
pre_col_list = prediction_df.columns[prediction_df.columns.str.startswith('prediction_')]
pre_col_list
prediction_df[PREDICTION_NAME] = prediction_df[pre_col_list].mean(axis=1)
print('correlarion:',correlation(prediction_df[PREDICTION_NAME], tournament_data[TARGET_NAME]))

n_of_features:  [[0.0007, 13]]
correlarion: 0.0002710617722087143


In [9]:
pred = pd.DataFrame()
pred["id"] = prediction_df['id']
pred["prediction"] = prediction_df['prediction']
pred.to_csv("./sub/predictions_xg_std_n11_seed_ave.csv", index=False)

In [10]:
median = pred['prediction'].median() *2
median

1.0005358457565308

In [11]:
pred2 = pd.DataFrame()
pred2["id"] = prediction_df['id']
pred2["prediction"] = median -prediction_df['prediction']
pred2.to_csv("./sub/predictions2_xg_std_n11_seed_ave.csv", index=False)

2021-06-06 06:55:31,894 INFO numexpr.utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-06-06 06:55:31,895 INFO numexpr.utils: NumExpr defaulting to 8 threads.


### 負データはPred2で保存。Corrが負ならModel1,3は負データ。Model2逆 目安は0.022~0.04

In [12]:
pred2.describe()

,prediction
count,1.736405e+06
mean,5.003251e-01
std,4.972872e-03
min,4.784310e-01
25%,4.968327e-01
50%,5.002679e-01
75%,5.037872e-01
max,5.189581e-01


In [13]:
pred.describe()

,prediction
count,1.736405e+06
mean,5.002106e-01
std,4.972872e-03
min,4.815778e-01
25%,4.967486e-01
50%,5.002679e-01
75%,5.037032e-01
max,5.221049e-01


In [14]:
pred2

,id,prediction
0,n0003aa52cab36c2,0.504852
1,n000920ed083903f,0.507622
2,n0038e640522c4a6,0.494481
3,n004ac94a87dc54b,0.493649
4,n0052fe97ea0c05f,0.500324
...,...,...
1736400,nffb5c07977c1523,0.499088
1736401,nffba1986e9d2901,0.492479
1736402,nffbf43467e75e64,0.505634
1736403,nffd9381ffdb9297,0.502092


In [15]:
pred

,id,prediction
0,n0003aa52cab36c2,0.495684
1,n000920ed083903f,0.492914
2,n0038e640522c4a6,0.506055
3,n004ac94a87dc54b,0.506887
4,n0052fe97ea0c05f,0.500212
...,...,...
1736400,nffb5c07977c1523,0.501447
1736401,nffba1986e9d2901,0.508057
1736402,nffbf43467e75e64,0.494902
1736403,nffd9381ffdb9297,0.498444


In [16]:
if correlation(pd.Series(prediction), tournament_data[TARGET_NAME])>0: #0.022
    submission_id = napi.upload_predictions(
    "./sub/predictions_xg_std_n11_seed_ave.csv", model_id= model_id)
    submission_id = napi.upload_predictions(
    "./sub/predictions2_xg_std_n11_seed_ave.csv", model_id= model_2)
    submission_id = napi.upload_predictions(
    "./sub/predictions_xg_std_n11_seed_ave.csv", model_id= model_3)
    print('uploaded prediction to KMTK49')
else:
    submission_id = napi.upload_predictions(
    "./sub/predictions2_xg_std_n11_seed_ave.csv", model_id= model_id)
    submission_id = napi.upload_predictions(
    "./sub/predictions_xg_std_n11_seed_ave.csv", model_id= model_2)
    submission_id = napi.upload_predictions(
    "./sub/predictions2_xg_std_n11_seed_ave.csv", model_id= model_3)
    print('uploaded prediction2 to KMTK49')

2021-06-06 06:55:37,043 INFO numerapi.base_api: uploading predictions...
2021-06-06 06:56:05,499 INFO numerapi.base_api: uploading predictions...
2021-06-06 06:56:40,130 INFO numerapi.base_api: uploading predictions...


uploaded prediction to KMTK49


In [17]:
print('correlation:',correlation(prediction_df[PREDICTION_NAME], tournament_data[TARGET_NAME]))

correlation: 0.0002710617722087143
